# 1. Goodbooks-10k 
- Link : https://www.kaggle.com/zygmunt/goodbooks-10k

In [1]:
import pandas as pd
import numpy as np
import plotnine 
from plotnine import *
import os, sys, gc
from tqdm.notebook import tqdm

In [2]:
# 경로의 경우 각자의 환경에 맞게 설정해주면 됩니다. 
path = '../input/t-academy-recommendation2/books/'

- books.csv : 책의 메타정보 
- book_tags.csv : 책-테그의 매핑정보 
- ratings.csv : 사용자가 책에 대해 점수를 준 평점정보 
- tags.csv : 테그의 정보 
- to_read.csv : 사용자가 읽으려고 기록해둔 책 (장바구니) 

In [4]:
books = pd.read_csv(path + "books.csv")
book_tags = pd.read_csv(path + "book_tags.csv")
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")
tags = pd.read_csv(path + "tags.csv")
to_read = pd.read_csv(path + "to_read.csv")

In [5]:
train['book_id'] = train['book_id'].astype(str)
test['book_id'] = test['book_id'].astype(str)
books['book_id'] = books['book_id'].astype(str)

In [6]:
sol = test.groupby(['user_id'])['book_id'].agg({'unique'}).reset_index()
gt = {}
for user in tqdm(sol['user_id'].unique()): 
    gt[user] = list(sol[sol['user_id'] == user]['unique'].values[0])

## 한 사람당 100권의 책을 추천해주는 상황 

In [7]:
rec_df = pd.DataFrame()
rec_df['user_id'] = train['user_id'].unique()

## Baseline 
- 통계기반의 모델 

In [8]:
books.sort_values(by='books_count', ascending=False)[0:5]

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
9,10,1885,1885,3060926,3455,679783261,9.780680e+12,Jane Austen,1813.0,Pride and Prejudice,...,2035490,2191465,49152,54700,86485,284852,609755,1155673,https://images.gr-assets.com/books/1320399351m...,https://images.gr-assets.com/books/1320399351s...
1724,1725,6324090,6324090,55548884,3296,NaN,NaN,"Lewis Carroll, John Tenniel",1865.0,NaN,...,30426,85031,5047,1724,4863,18094,27848,32502,https://images.gr-assets.com/books/1391204048m...,https://images.gr-assets.com/books/1391204048s...
70,71,18490,18490,4836639,2618,141439475,9.780141e+12,"Mary Wollstonecraft Shelley, Percy Bysshe Shel...",1818.0,"Frankenstein; or, The Modern Prometheus",...,808589,883563,19993,29366,74142,231022,300495,248538,https://images.gr-assets.com/books/1381512375m...,https://images.gr-assets.com/books/1381512375s...
292,293,295,295,3077988,2573,753453800,9.780753e+12,Robert Louis Stevenson,1882.0,Treasure Island,...,268012,310515,7917,5349,19127,86487,114968,84584,https://images.gr-assets.com/books/1485248909m...,https://images.gr-assets.com/books/1485248909s...
42,43,10210,10210,2977639,2568,142437204,9.780142e+12,"Charlotte Brontë, Michael Mason",1847.0,Jane Eyre,...,1198557,1286135,31212,35132,64274,212294,400214,574221,https://images.gr-assets.com/books/1327867269m...,https://images.gr-assets.com/books/1327867269s...


In [9]:
popular_rec_model = books.sort_values(by='books_count', ascending=False)['book_id'].values[0:500]

In [10]:
total_rec_list = {}
for user in tqdm(rec_df['user_id'].unique()):
    rec_list = []
    for rec in popular_rec_model[0:200]: 
        rec_list.append(rec)
    total_rec_list[user] = rec_list

In [11]:
import six
import math

# https://github.com/kakao-arena/brunch-article-recommendation/blob/master/evaluate.py

class evaluate():
    def __init__(self, recs, gt, topn=100):
        self.recs = recs
        self.gt = gt 
        self.topn = topn 
        
    def _ndcg(self):
        Q, S = 0.0, 0.0
        for u, seen in six.iteritems(self.gt):
            seen = list(set(seen))
            rec = self.recs.get(u, [])
            if not rec or len(seen) == 0:
                continue

            dcg = 0.0
            idcg = sum([1.0 / math.log(i + 2, 2) for i in range(min(len(seen), len(rec)))])
            for i, r in enumerate(rec):
                if r not in seen:
                    continue
                rank = i + 1
                dcg += 1.0 / math.log(rank + 1, 2)
            ndcg = dcg / idcg
            S += ndcg
            Q += 1
        return S / Q


    def _map(self):
        n, ap = 0.0, 0.0
        for u, seen in six.iteritems(self.gt):
            seen = list(set(seen))
            rec = self.recs.get(u, [])
            if not rec or len(seen) == 0:
                continue

            _ap, correct = 0.0, 0.0
            for i, r in enumerate(rec):
                if r in seen:
                    correct += 1
                    _ap += (correct / (i + 1.0))
            _ap /= min(len(seen), len(rec))
            ap += _ap
            n += 1.0
        return ap / n


    def _entropy_diversity(self):
        sz = float(len(self.recs)) * self.topn
        freq = {}
        for u, rec in six.iteritems(self.recs):
            for r in rec:
                freq[r] = freq.get(r, 0) + 1
        ent = -sum([v / sz * math.log(v / sz) for v in six.itervalues(freq)])
        return ent
    
    def _evaluate(self):
        print('MAP@%s: %s' % (self.topn, self._map()))
        print('NDCG@%s: %s' % (self.topn, self._ndcg()))
        print('EntDiv@%s: %s' % (self.topn, self._entropy_diversity()))

In [12]:
evaluate_func = evaluate(recs=total_rec_list, gt = gt, topn=200)
evaluate_func._evaluate()

MAP@200: 0.00015443987007568514
NDCG@200: 0.0017911738188034422
EntDiv@200: 5.298317366548021


## Baseline 응용 
- 많이 글은 글중에서도 평점이 높은 글들을 우선적으로 추천 
- 내가 좋아하는 작가의 글을 우선적으로 추천 
- 장바구니에 담긴 글과 작가의 글을 우선적으로 추천 
- 읽은 글의 시리즈글이 나오면 추천 (해리포터 마법사의 돌 -> 비밀의 방)
- 최신의 글을 추천

In [13]:
train = pd.merge(train, books[['book_id', 'authors', 'ratings_count']], how='left', on='book_id')

In [14]:
agg = train.groupby(['user_id','authors'])['authors'].agg({'count'}).reset_index()
agg = agg.sort_values(by='count', ascending=False)
agg.head()

,user_id,authors,count
19036,30944,Bill Bryson,7
8420,13544,Bill Bryson,6
26719,45269,Bill Bryson,6
12906,20845,Thomas Pynchon,6
8233,13282,Bill Bryson,6


In [20]:
author_books = books[['book_id', 'authors', 'ratings_count']].sort_values(by=['authors', 'ratings_count'], ascending=[True, False])
author_books = author_books.reset_index(drop=True)

author_books.head()

,book_id,authors,ratings_count
0,101911,A. Manette Ansay,23966
1,16081754,A. Meredith Walters,24615
2,16102880,A. Meredith Walters,24287
3,99107,"A.A. Milne, Ernest H. Shepard",207550
4,776407,"A.A. Milne, Ernest H. Shepard",69102


In [21]:
author_rec_model = agg.merge(author_books, how='left', on=['authors'])

In [22]:
author_rec_model.head()

,user_id,authors,count,book_id,ratings_count
0,30944,Bill Bryson,7,9791,250192
1,30944,Bill Bryson,7,21,191535
2,30944,Bill Bryson,7,28,66947
3,30944,Bill Bryson,7,24,58688
4,30944,Bill Bryson,7,7507825,55296


In [23]:
author_rec_model[author_rec_model['user_id'] == 30944]['book_id'].values

array(['9791', '21', '28', '24', '7507825', '25', '10538', '27', '26',
       '17262366', '135611', '29', '25489012', '10541', '50', '141372',
       '2915', '350', '17214', '16690', '353', '7171856', '17208',
       '20417', '16685', '348', '175319', '175324', '355', '125963',
       '125956', '125955', '456028', '4789801', '2142', '456057',
       '125959', '125958', '1866046', '1431', '1430', '1427', '16054811',
       '1715', '122'], dtype=object)

In [24]:
total_rec_list = {}
for user in tqdm(rec_df['user_id'].unique()):
    rec_list = []
    author_rec_model_ = author_rec_model[author_rec_model['user_id'] == user]['book_id'].values
    for rec in author_rec_model_: 
        rec_list.append(rec)
    
    if len(rec_list) < 200:
        for i in popular_rec_model[0:200]:
            rec_list.append(rec)
        
    total_rec_list[user] = rec_list[0:200]

In [25]:
evaluate_func = evaluate(recs=total_rec_list, gt = gt, topn=200)
evaluate_func._evaluate()

MAP@200: 0.010888309165191986
NDCG@200: 0.004074971802873805
EntDiv@200: 5.844823927882108


## 후처리
- 내가 읽은 책은 추천해주면 안됨 
- 내가 읽은 언어와 맞는 책을 추천해줘야함 

In [26]:
# 내가 읽은 책의 목록을 추출 
read_list = train.groupby(['user_id'])['book_id'].agg({'unique'}).reset_index()
read_list.head()

,user_id,unique
0,1,[4893]
1,2,[8855]
2,3,[9049]
3,4,[3273]
4,5,"[4829, 6703]"


In [27]:
total_rec_list = {}
for user in tqdm(rec_df['user_id'].unique()):
    rec_list = []
    author_rec_model_ = author_rec_model[author_rec_model['user_id'] == user]['book_id'].values
    seen = read_list[read_list['user_id'] == user]['unique'].values[0]
    for rec in author_rec_model_: 
        if rec not in seen:
            rec_list.append(rec)
    
    if len(rec_list) < 200:
        for i in popular_rec_model[0:200]:
            if rec not in seen:
                rec_list.append(rec)

    total_rec_list[user] = rec_list[0:200]

In [28]:
evaluate_func = evaluate(recs=total_rec_list, gt = gt, topn=200)
evaluate_func._evaluate()

MAP@200: 0.011817742695221473
NDCG@200: 0.004577031690605801
EntDiv@200: 5.097773423502677


In [29]:
# 내가 읽을 수 있는 언어의 목록을 추출 
## User에 대한 메타정보가 있으면 쉽게 추출가능하지만, 현재는 없으므로 직접 생성 
## Ratings에서 읽은 책들의 언어를 전부 수집해서 해당 언어의 책들을 가능한 언어로 설정 
language = pd.merge(train, books[['book_id', 'language_code']], how='left', on='book_id')

In [30]:
language_list = language.groupby(['user_id'])['language_code'].agg({'unique'}).reset_index()
language_list.head()

,user_id,unique
0,1,[nan]
1,2,[nan]
2,3,[nan]
3,4,[en-US]
4,5,[nan]
